In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D, Dropout
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = Dropout(0.1) (c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = Dropout(0.1) (c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/batchdsdropustest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 18s 179ms/step - loss: 5.2245 - binary_crossentropy: 1.3620 - jaccard_coef_int: 0.1737 - binary_accuracy: 0.7913 - val_loss: 8.5482 - val_binary_crossentropy: 3.1889 - val_jaccard_coef_int: 0.0058 - val_binary_accuracy: 0.7984
Epoch 2/50
100/100 [==============================] - 11s 115ms/step - loss: 4.2730 - binary_crossentropy: 2.0434 - jaccard_coef_int: 0.2193 - binary_accuracy: 0.8315 - val_loss: 12.4101 - val_binary_crossentropy: 3.1920 - val_jaccard_coef_int: 8.9892e-16 - val_binary_accuracy: 0.8009
Epoch 3/50
100/100 [==============================] - 11s 114ms/step - loss: 4.1174 - binary_crossentropy: 2.1849 - jaccard_coef_int: 0.2164 - binary_accuracy: 0.8411 - val_loss: 9.7279 - val_binary_crossentropy: 3.2995 - val_jaccard_coef_int: 0.0017 - val_binary_accuracy: 0.7915
Epoch 4/50
100/100 [==============================] - 12s 116ms/step - loss: 3.9957 - binary_crossentropy: 2.2547 - jaccard_coef_int: 0.2387 - binary_ac

100/100 [==============================] - 11s 115ms/step - loss: 3.3562 - binary_crossentropy: 2.1885 - jaccard_coef_int: 0.3321 - binary_accuracy: 0.8603 - val_loss: 3.6550 - val_binary_crossentropy: 2.6280 - val_jaccard_coef_int: 0.3842 - val_binary_accuracy: 0.8316
Epoch 31/50
100/100 [==============================] - 12s 116ms/step - loss: 3.3472 - binary_crossentropy: 2.1976 - jaccard_coef_int: 0.3326 - binary_accuracy: 0.8607 - val_loss: 4.0386 - val_binary_crossentropy: 2.8745 - val_jaccard_coef_int: 0.3304 - val_binary_accuracy: 0.8169
Epoch 32/50
100/100 [==============================] - 11s 115ms/step - loss: 3.2668 - binary_crossentropy: 2.1041 - jaccard_coef_int: 0.3312 - binary_accuracy: 0.8661 - val_loss: 4.3837 - val_binary_crossentropy: 3.3053 - val_jaccard_coef_int: 0.3836 - val_binary_accuracy: 0.7868
Epoch 33/50
100/100 [==============================] - 12s 115ms/step - loss: 3.3841 - binary_crossentropy: 2.2540 - jaccard_coef_int: 0.3396 - binary_accuracy: 0.856

Epoch 16/50
100/100 [==============================] - 11s 112ms/step - loss: 2.3225 - binary_crossentropy: 0.1332 - jaccard_coef_int: 0.1229 - binary_accuracy: 0.9902 - val_loss: 2.8496 - val_binary_crossentropy: 0.2430 - val_jaccard_coef_int: 0.0770 - val_binary_accuracy: 0.9821
Epoch 17/50
100/100 [==============================] - 11s 111ms/step - loss: 2.3611 - binary_crossentropy: 0.1395 - jaccard_coef_int: 0.1211 - binary_accuracy: 0.9899 - val_loss: 3.0283 - val_binary_crossentropy: 0.2171 - val_jaccard_coef_int: 0.0618 - val_binary_accuracy: 0.9850
Epoch 18/50
100/100 [==============================] - 11s 112ms/step - loss: 2.3037 - binary_crossentropy: 0.1352 - jaccard_coef_int: 0.1289 - binary_accuracy: 0.9902 - val_loss: 2.8165 - val_binary_crossentropy: 0.2515 - val_jaccard_coef_int: 0.0804 - val_binary_accuracy: 0.9816
Epoch 19/50
100/100 [==============================] - 11s 113ms/step - loss: 4.1897 - binary_crossentropy: 0.2510 - jaccard_coef_int: 0.1050 - binary_acc

...

100/100 [==============================] - 11s 113ms/step - loss: 3.3212 - binary_crossentropy: 1.1510 - jaccard_coef_int: 0.1345 - binary_accuracy: 0.9190 - val_loss: 5.9295 - val_binary_crossentropy: 0.9533 - val_jaccard_coef_int: 0.0070 - val_binary_accuracy: 0.9395
Epoch 31/50
100/100 [==============================] - 11s 114ms/step - loss: 3.2282 - binary_crossentropy: 1.1074 - jaccard_coef_int: 0.1417 - binary_accuracy: 0.9217 - val_loss: 3.9077 - val_binary_crossentropy: 1.2293 - val_jaccard_coef_int: 0.0840 - val_binary_accuracy: 0.9108
Epoch 32/50
100/100 [==============================] - 11s 114ms/step - loss: 3.2125 - binary_crossentropy: 1.1277 - jaccard_coef_int: 0.1442 - binary_accuracy: 0.9215 - val_loss: 4.6005 - val_binary_crossentropy: 0.9750 - val_jaccard_coef_int: 0.0281 - val_binary_accuracy: 0.9355
Epoch 33/50
100/100 [==============================] - 11s 114ms/step - loss: 3.1789 - binary_crossentropy: 1.1255 - jaccard_coef_int: 0.1475 - binary_accuracy: 0.922

Epoch 7/50
100/100 [==============================] - 11s 114ms/step - loss: 4.4044 - binary_crossentropy: 0.7804 - jaccard_coef_int: 0.0572 - binary_accuracy: 0.9079 - val_loss: 5.2666 - val_binary_crossentropy: 0.9107 - val_jaccard_coef_int: 0.0127 - val_binary_accuracy: 0.9259
Epoch 8/50
100/100 [==============================] - 11s 114ms/step - loss: 4.2410 - binary_crossentropy: 0.8302 - jaccard_coef_int: 0.0637 - binary_accuracy: 0.9095 - val_loss: 5.0950 - val_binary_crossentropy: 0.8971 - val_jaccard_coef_int: 0.0234 - val_binary_accuracy: 0.9114
Epoch 9/50
100/100 [==============================] - 11s 115ms/step - loss: 4.1777 - binary_crossentropy: 0.8475 - jaccard_coef_int: 0.0660 - binary_accuracy: 0.9113 - val_loss: 5.8125 - val_binary_crossentropy: 1.1360 - val_jaccard_coef_int: 0.0708 - val_binary_accuracy: 0.8038
Epoch 10/50
100/100 [==============================] - 11s 113ms/step - loss: 4.1373 - binary_crossentropy: 0.8652 - jaccard_coef_int: 0.0683 - binary_accura

100/100 [==============================] - 11s 113ms/step - loss: 3.5856 - binary_crossentropy: 1.1098 - jaccard_coef_int: 0.0978 - binary_accuracy: 0.9222 - val_loss: 3.8893 - val_binary_crossentropy: 1.3232 - val_jaccard_coef_int: 0.0893 - val_binary_accuracy: 0.9081
Epoch 37/50
100/100 [==============================] - 12s 116ms/step - loss: 3.5936 - binary_crossentropy: 1.1171 - jaccard_coef_int: 0.0969 - binary_accuracy: 0.9221 - val_loss: 4.4475 - val_binary_crossentropy: 1.1177 - val_jaccard_coef_int: 0.0395 - val_binary_accuracy: 0.9238
Epoch 38/50
100/100 [==============================] - 12s 116ms/step - loss: 3.5668 - binary_crossentropy: 1.0959 - jaccard_coef_int: 0.0974 - binary_accuracy: 0.9239 - val_loss: 3.9265 - val_binary_crossentropy: 1.4875 - val_jaccard_coef_int: 0.1146 - val_binary_accuracy: 0.8905
Epoch 39/50
100/100 [==============================] - 11s 113ms/step - loss: 3.5860 - binary_crossentropy: 1.1249 - jaccard_coef_int: 0.0986 - binary_accuracy: 0.921

Epoch 13/50
100/100 [==============================] - 9s 88ms/step - loss: 2.5601 - binary_crossentropy: 0.1476 - jaccard_coef_int: 0.1776 - binary_accuracy: 0.9889 - val_loss: 5.6904 - val_binary_crossentropy: 0.0310 - val_jaccard_coef_int: 0.5660 - val_binary_accuracy: 0.9979
Epoch 14/50
100/100 [==============================] - 9s 88ms/step - loss: 2.7279 - binary_crossentropy: 0.1519 - jaccard_coef_int: 0.1565 - binary_accuracy: 0.9889 - val_loss: 5.3373 - val_binary_crossentropy: 0.0327 - val_jaccard_coef_int: 0.5536 - val_binary_accuracy: 0.9978
Epoch 15/50
100/100 [==============================] - 9s 87ms/step - loss: 2.6738 - binary_crossentropy: 0.1482 - jaccard_coef_int: 0.1706 - binary_accuracy: 0.9893 - val_loss: 4.5838 - val_binary_crossentropy: 0.0272 - val_jaccard_coef_int: 0.5728 - val_binary_accuracy: 0.9981
Epoch 16/50
100/100 [==============================] - 9s 89ms/step - loss: 2.9492 - binary_crossentropy: 0.1753 - jaccard_coef_int: 0.1419 - binary_accuracy: 0

100/100 [==============================] - 9s 87ms/step - loss: 3.1135 - binary_crossentropy: 0.3723 - jaccard_coef_int: 0.1256 - binary_accuracy: 0.9746 - val_loss: 11.9872 - val_binary_crossentropy: 0.0098 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 43/50
100/100 [==============================] - 9s 89ms/step - loss: 3.0833 - binary_crossentropy: 0.2596 - jaccard_coef_int: 0.1279 - binary_accuracy: 0.9820 - val_loss: 2.9836 - val_binary_crossentropy: 0.0099 - val_jaccard_coef_int: 0.6250 - val_binary_accuracy: 0.9994
Epoch 44/50
100/100 [==============================] - 9s 87ms/step - loss: 2.7521 - binary_crossentropy: 0.2255 - jaccard_coef_int: 0.1369 - binary_accuracy: 0.9847 - val_loss: 5.1951 - val_binary_crossentropy: 0.0106 - val_jaccard_coef_int: 0.6309 - val_binary_accuracy: 0.9993
Epoch 45/50
100/100 [==============================] - 9s 87ms/step - loss: 2.6655 - binary_crossentropy: 0.2282 - jaccard_coef_int: 0.1401 - binary_accuracy: 0.9848 - val

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchdsdropustest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 16s 157ms/step - loss: 5.4421 - binary_crossentropy: 2.2426 - jaccard_coef_int: 0.2716 - binary_accuracy: 0.7589 - val_loss: 4.8021 - val_binary_crossentropy: 3.1543 - val_jaccard_coef_int: 0.3527 - val_binary_accuracy: 0.7668
Epoch 2/50
100/100 [==============================] - 11s 111ms/step - loss: 4.4050 - binary_crossentropy: 2.8655 - jaccard_coef_int: 0.3876 - binary_accuracy: 0.7845 - val_loss: 4.6182 - val_binary_crossentropy: 2.6909 - val_jaccard_coef_int: 0.2366 - val_binary_accuracy: 0.8012
Epoch 3/50
100/100 [==============================] - 11s 110ms/step - loss: 4.3296 - binary_crossentropy: 2.9740 - jaccard_coef_int: 0.3973 - binary_accuracy: 0.7876 - val_loss: 4.2998 - val_binary_crossentropy: 2.7841 - val_jaccard_coef_int: 0.2973 - val_binary_accuracy: 0.8072
Epoch 4/50
100/100 [==============================] - 11s 111ms/step - loss: 4.3589 - binary_crossentropy: 3.0802 - jaccard_coef_int: 0.3889 - binary_accurac

100/100 [==============================] - 11s 111ms/step - loss: 3.2806 - binary_crossentropy: 2.5291 - jaccard_coef_int: 0.5080 - binary_accuracy: 0.8376 - val_loss: 4.0135 - val_binary_crossentropy: 3.0513 - val_jaccard_coef_int: 0.4129 - val_binary_accuracy: 0.8051
Epoch 31/50
100/100 [==============================] - 11s 111ms/step - loss: 3.3373 - binary_crossentropy: 2.5762 - jaccard_coef_int: 0.5046 - binary_accuracy: 0.8344 - val_loss: 5.6161 - val_binary_crossentropy: 4.4590 - val_jaccard_coef_int: 0.3387 - val_binary_accuracy: 0.7180
Epoch 32/50
100/100 [==============================] - 11s 112ms/step - loss: 3.2295 - binary_crossentropy: 2.5001 - jaccard_coef_int: 0.5197 - binary_accuracy: 0.8393 - val_loss: 3.8968 - val_binary_crossentropy: 2.9042 - val_jaccard_coef_int: 0.4123 - val_binary_accuracy: 0.8124
Epoch 33/50
100/100 [==============================] - 11s 112ms/step - loss: 3.2610 - binary_crossentropy: 2.5135 - jaccard_coef_int: 0.5093 - binary_accuracy: 0.838

Epoch 7/50
100/100 [==============================] - 11s 113ms/step - loss: 4.2090 - binary_crossentropy: 1.9902 - jaccard_coef_int: 0.1243 - binary_accuracy: 0.8710 - val_loss: 7.0994 - val_binary_crossentropy: 3.7249 - val_jaccard_coef_int: 0.0358 - val_binary_accuracy: 0.7651
Epoch 8/50
100/100 [==============================] - 11s 112ms/step - loss: 4.1882 - binary_crossentropy: 1.9608 - jaccard_coef_int: 0.1240 - binary_accuracy: 0.8724 - val_loss: 4.7461 - val_binary_crossentropy: 2.4059 - val_jaccard_coef_int: 0.1102 - val_binary_accuracy: 0.8412
Epoch 9/50
100/100 [==============================] - 11s 113ms/step - loss: 4.0886 - binary_crossentropy: 1.8983 - jaccard_coef_int: 0.1277 - binary_accuracy: 0.8766 - val_loss: 6.0959 - val_binary_crossentropy: 3.4626 - val_jaccard_coef_int: 0.1027 - val_binary_accuracy: 0.7623
Epoch 10/50
100/100 [==============================] - 11s 113ms/step - loss: 4.1105 - binary_crossentropy: 1.9126 - jaccard_coef_int: 0.1257 - binary_accura

...

100/100 [==============================] - 11s 110ms/step - loss: 29.9430 - binary_crossentropy: 0.2227 - jaccard_coef_int: 0.0178 - binary_accuracy: 0.9862 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 14/50
100/100 [==============================] - 11s 112ms/step - loss: 31.4270 - binary_crossentropy: 0.2320 - jaccard_coef_int: 0.0107 - binary_accuracy: 0.9856 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 15/50
100/100 [==============================] - 11s 112ms/step - loss: 29.9496 - binary_crossentropy: 0.2343 - jaccard_coef_int: 0.0263 - binary_accuracy: 0.9855 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 16/50
100/100 [==============================] - 11s 112ms/step - loss: 31.1113 - binary_crossentropy: 0.2243 - jaccard_coef_int: 0.0165 - bin

100/100 [==============================] - 11s 111ms/step - loss: 30.2593 - binary_crossentropy: 0.2340 - jaccard_coef_int: 0.0158 - binary_accuracy: 0.9855 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 42/50
100/100 [==============================] - 11s 112ms/step - loss: 29.8267 - binary_crossentropy: 0.2158 - jaccard_coef_int: 0.0323 - binary_accuracy: 0.9866 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 43/50
100/100 [==============================] - 11s 111ms/step - loss: 31.6653 - binary_crossentropy: 0.2392 - jaccard_coef_int: 0.0087 - binary_accuracy: 0.9852 - val_loss: 31.8975 - val_binary_crossentropy: 0.1761 - val_jaccard_coef_int: 1.6701e-14 - val_binary_accuracy: 0.9891
Epoch 44/50
100/100 [==============================] - 11s 113ms/step - loss: 31.1727 - binary_crossentropy: 0.2345 - jaccard_coef_int: 0.0120 - bin

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99   1095748
        1.0       0.00      0.00      0.00     11452

avg / total       0.98      0.99      0.98   1107200



label: 99
{99: 1.0}
composition with label != 0
Y_train 0.10628019323671498 0.009019726247987117
composition with label != 0
Y_val 0.12857142857142856 0.012348214285714285
composition with label != 0
Y_test 0.09826589595375723 0.005773121387283237
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)
Epoch 1/50
100/100 [==============================] - 16s 160ms/step - loss: 5.9200 - binary_crossentropy: 0.1135 - jaccard_coef_int: 0.0022 - binary_accuracy: 0.9809 - val_loss: 5.3154 - val_binary_crossentropy: 0.1154 - val_jaccard_coef_int: 1.6524e-14 - val_binary_accuracy: 0.9877
Epoch 2/50
100/100 [==============================] - 11s 114ms/

Epoch 28/50
100/100 [==============================] - 11s 114ms/step - loss: 4.2777 - binary_crossentropy: 0.4572 - jaccard_coef_int: 0.0362 - binary_accuracy: 0.9703 - val_loss: 3.1555 - val_binary_crossentropy: 0.5370 - val_jaccard_coef_int: 0.0736 - val_binary_accuracy: 0.9661
Epoch 29/50
100/100 [==============================] - 11s 111ms/step - loss: 4.2675 - binary_crossentropy: 0.4543 - jaccard_coef_int: 0.0364 - binary_accuracy: 0.9708 - val_loss: 3.1413 - val_binary_crossentropy: 0.5306 - val_jaccard_coef_int: 0.0740 - val_binary_accuracy: 0.9666
Epoch 30/50
100/100 [==============================] - 11s 113ms/step - loss: 4.3598 - binary_crossentropy: 0.4347 - jaccard_coef_int: 0.0315 - binary_accuracy: 0.9720 - val_loss: 3.3102 - val_binary_crossentropy: 0.6061 - val_jaccard_coef_int: 0.0677 - val_binary_accuracy: 0.9616
Epoch 31/50
100/100 [==============================] - 11s 114ms/step - loss: 5.4286 - binary_crossentropy: 0.4335 - jaccard_coef_int: 0.0164 - binary_acc

In [16]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchdsdropustest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 16s 156ms/step - loss: 1.5419 - binary_crossentropy: 0.0966 - jaccard_coef_int: 0.6335 - binary_accuracy: 0.9762 - val_loss: 0.6390 - val_binary_crossentropy: 0.1417 - val_jaccard_coef_int: 0.6175 - val_binary_accuracy: 0.9903
Epoch 2/50
100/100 [==============================] - 11s 110ms/step - loss: 0.3868 - binary_crossentropy: 0.0472 - jaccard_coef_int: 0.7460 - binary_accuracy: 0.9955 - val_loss: 0.5651 - val_binary_crossentropy: 0.1235 - val_jaccard_coef_int: 0.6572 - val_binary_accuracy: 0.9910
Epoch 3/50
100/100 [==============================] - 11s 110ms/step - loss: 0.4036 - binary_crossentropy: 0.0556 - jaccard_coef_int: 0.7413 - binary_accuracy: 0.9953 - val_loss: 0.5717 - val_binary_crossentropy: 0.1306 - val_jaccard_coef_int: 0.6534 - val_binary_accuracy: 0.9909
Epoch 4/50
100/100 [==============================] - 11s 111ms/step - loss: 0.3594 - binary_crossentropy: 0.0602 - jaccard_coef_int: 0.7601 - binary_accurac

100/100 [==============================] - 11s 111ms/step - loss: 0.3415 - binary_crossentropy: 0.0644 - jaccard_coef_int: 0.7652 - binary_accuracy: 0.9958 - val_loss: 0.5377 - val_binary_crossentropy: 0.1366 - val_jaccard_coef_int: 0.6716 - val_binary_accuracy: 0.9913
Epoch 31/50
100/100 [==============================] - 11s 111ms/step - loss: 0.3617 - binary_crossentropy: 0.0766 - jaccard_coef_int: 0.7581 - binary_accuracy: 0.9950 - val_loss: 0.5434 - val_binary_crossentropy: 0.1376 - val_jaccard_coef_int: 0.6683 - val_binary_accuracy: 0.9912
Epoch 32/50
100/100 [==============================] - 11s 110ms/step - loss: 0.3708 - binary_crossentropy: 0.0681 - jaccard_coef_int: 0.7475 - binary_accuracy: 0.9956 - val_loss: 0.5443 - val_binary_crossentropy: 0.1379 - val_jaccard_coef_int: 0.6679 - val_binary_accuracy: 0.9913
Epoch 33/50
100/100 [==============================] - 11s 108ms/step - loss: 0.3463 - binary_crossentropy: 0.0631 - jaccard_coef_int: 0.7611 - binary_accuracy: 0.995

Epoch 7/50
100/100 [==============================] - 11s 112ms/step - loss: 4.6957 - binary_crossentropy: 3.2164 - jaccard_coef_int: 0.2828 - binary_accuracy: 0.7859 - val_loss: 5.0703 - val_binary_crossentropy: 3.3264 - val_jaccard_coef_int: 0.1865 - val_binary_accuracy: 0.7879
Epoch 8/50
100/100 [==============================] - 11s 113ms/step - loss: 4.6489 - binary_crossentropy: 3.2190 - jaccard_coef_int: 0.2931 - binary_accuracy: 0.7865 - val_loss: 5.0734 - val_binary_crossentropy: 3.0473 - val_jaccard_coef_int: 0.1796 - val_binary_accuracy: 0.7899
Epoch 9/50
100/100 [==============================] - 11s 111ms/step - loss: 4.6241 - binary_crossentropy: 3.1985 - jaccard_coef_int: 0.2959 - binary_accuracy: 0.7875 - val_loss: 4.6750 - val_binary_crossentropy: 3.2369 - val_jaccard_coef_int: 0.2896 - val_binary_accuracy: 0.7856
Epoch 10/50
100/100 [==============================] - 11s 113ms/step - loss: 4.5947 - binary_crossentropy: 3.1920 - jaccard_coef_int: 0.3014 - binary_accura

...

Epoch 34/50
100/100 [==============================] - 11s 115ms/step - loss: 5.3397 - binary_crossentropy: 1.8339 - jaccard_coef_int: 0.0300 - binary_accuracy: 0.8856 - val_loss: 35.1450 - val_binary_crossentropy: 1.3722 - val_jaccard_coef_int: 2.1192e-15 - val_binary_accuracy: 0.9149
Epoch 35/50
100/100 [==============================] - 11s 110ms/step - loss: 5.3446 - binary_crossentropy: 1.8007 - jaccard_coef_int: 0.0294 - binary_accuracy: 0.8865 - val_loss: 35.1450 - val_binary_crossentropy: 1.3722 - val_jaccard_coef_int: 2.1192e-15 - val_binary_accuracy: 0.9149
Epoch 36/50
100/100 [==============================] - 11s 113ms/step - loss: 5.3337 - binary_crossentropy: 1.8295 - jaccard_coef_int: 0.0300 - binary_accuracy: 0.8860 - val_loss: 35.1450 - val_binary_crossentropy: 1.3722 - val_jaccard_coef_int: 2.1192e-15 - val_binary_accuracy: 0.9149
Epoch 37/50
100/100 [==============================] - 11s 111ms/step - loss: 5.3351 - binary_crossentropy: 1.8277 - jaccard_coef_int: 0.02

100/100 [==============================] - 14s 138ms/step - loss: 2.9590 - binary_crossentropy: 0.1398 - jaccard_coef_int: 0.1414 - binary_accuracy: 0.9857 - val_loss: 28.7754 - val_binary_crossentropy: 0.0010 - val_jaccard_coef_int: 0.7875 - val_binary_accuracy: 0.9999
Epoch 11/50
100/100 [==============================] - 11s 110ms/step - loss: 4.4324 - binary_crossentropy: 0.3297 - jaccard_coef_int: 0.0756 - binary_accuracy: 0.9621 - val_loss: 32.5733 - val_binary_crossentropy: 0.0341 - val_jaccard_coef_int: 1.4151e-14 - val_binary_accuracy: 0.9853
Epoch 12/50
100/100 [==============================] - 11s 113ms/step - loss: 3.6655 - binary_crossentropy: 0.1296 - jaccard_coef_int: 0.0881 - binary_accuracy: 0.9822 - val_loss: 32.0199 - val_binary_crossentropy: 0.0350 - val_jaccard_coef_int: 2.2378e-14 - val_binary_accuracy: 0.9896
Epoch 13/50
100/100 [==============================] - 11s 111ms/step - loss: 3.1454 - binary_crossentropy: 0.1199 - jaccard_coef_int: 0.1288 - binary_accu

Epoch 39/50
100/100 [==============================] - 11s 110ms/step - loss: 3.4855 - binary_crossentropy: 0.0777 - jaccard_coef_int: 0.1332 - binary_accuracy: 0.9909 - val_loss: 28.8733 - val_binary_crossentropy: 0.0115 - val_jaccard_coef_int: 0.3500 - val_binary_accuracy: 0.9985
Epoch 40/50
100/100 [==============================] - 11s 111ms/step - loss: 2.6883 - binary_crossentropy: 0.0920 - jaccard_coef_int: 0.1536 - binary_accuracy: 0.9901 - val_loss: 29.6971 - val_binary_crossentropy: 0.0451 - val_jaccard_coef_int: 0.0625 - val_binary_accuracy: 0.9963
Epoch 41/50
100/100 [==============================] - 11s 112ms/step - loss: 3.0130 - binary_crossentropy: 0.0953 - jaccard_coef_int: 0.1389 - binary_accuracy: 0.9897 - val_loss: 29.3994 - val_binary_crossentropy: 0.0165 - val_jaccard_coef_int: 0.2625 - val_binary_accuracy: 0.9974
Epoch 42/50
100/100 [==============================] - 11s 108ms/step - loss: 3.0311 - binary_crossentropy: 0.1010 - jaccard_coef_int: 0.1437 - binary_

Epoch 16/50
100/100 [==============================] - 11s 110ms/step - loss: 2.0990 - binary_crossentropy: 0.0819 - jaccard_coef_int: 0.1695 - binary_accuracy: 0.9940 - val_loss: 4.4577 - val_binary_crossentropy: 0.3457 - val_jaccard_coef_int: 0.0178 - val_binary_accuracy: 0.9739
Epoch 17/50
100/100 [==============================] - 11s 109ms/step - loss: 8.0645 - binary_crossentropy: 0.7940 - jaccard_coef_int: 0.0956 - binary_accuracy: 0.9479 - val_loss: 2.3393 - val_binary_crossentropy: 0.0371 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 18/50
100/100 [==============================] - 11s 109ms/step - loss: 24.8611 - binary_crossentropy: 0.0639 - jaccard_coef_int: 0.0235 - binary_accuracy: 0.9960 - val_loss: 2.3393 - val_binary_crossentropy: 0.0371 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 19/50
100/100 [==============================] - 11s 108ms/step - loss: 24.7983 - binary_crossentropy: 0.0644 - jaccard_coef_int: 0.0338 - binary_a

Epoch 45/50
100/100 [==============================] - 11s 109ms/step - loss: 27.0116 - binary_crossentropy: 0.0631 - jaccard_coef_int: 0.0223 - binary_accuracy: 0.9961 - val_loss: 2.3393 - val_binary_crossentropy: 0.0371 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 46/50
100/100 [==============================] - 11s 108ms/step - loss: 25.3882 - binary_crossentropy: 0.0626 - jaccard_coef_int: 0.0280 - binary_accuracy: 0.9961 - val_loss: 2.3393 - val_binary_crossentropy: 0.0371 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 47/50
100/100 [==============================] - 11s 109ms/step - loss: 25.3198 - binary_crossentropy: 0.0596 - jaccard_coef_int: 0.0337 - binary_accuracy: 0.9963 - val_loss: 2.3393 - val_binary_crossentropy: 0.0371 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9977
Epoch 48/50
100/100 [==============================] - 11s 112ms/step - loss: 25.7110 - binary_crossentropy: 0.0639 - jaccard_coef_int: 0.0153 - binary

In [15]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/batchdsdropustest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        del ts_gen
        del X_train, X_val, X_test, Y_train, Y_val, Y_test
        del Y_predict
        K.clear_session()          
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100, epochs=50)`


Epoch 1/50
100/100 [==============================] - 14s 144ms/step - loss: 3.3939 - binary_crossentropy: 0.4319 - jaccard_coef_int: 0.2372 - binary_accuracy: 0.9009 - val_loss: 8.5760 - val_binary_crossentropy: 1.0954 - val_jaccard_coef_int: 2.4021e-04 - val_binary_accuracy: 0.9120
Epoch 2/50
100/100 [==============================] - 9s 88ms/step - loss: 1.9782 - binary_crossentropy: 0.4621 - jaccard_coef_int: 0.4544 - binary_accuracy: 0.9273 - val_loss: 3.8048 - val_binary_crossentropy: 0.8683 - val_jaccard_coef_int: 0.0873 - val_binary_accuracy: 0.9143
Epoch 3/50
100/100 [==============================] - 9s 89ms/step - loss: 1.8545 - binary_crossentropy: 0.5346 - jaccard_coef_int: 0.4826 - binary_accuracy: 0.9307 - val_loss: 2.5090 - val_binary_crossentropy: 0.9273 - val_jaccard_coef_int: 0.4302 - val_binary_accuracy: 0.8969
Epoch 4/50
100/100 [==============================] - 9s 93ms/step - loss: 1.6558 - binary_crossentropy: 0.5160 - jaccard_coef_int: 0.5170 - binary_accuracy:

100/100 [==============================] - 9s 88ms/step - loss: 0.7130 - binary_crossentropy: 0.2795 - jaccard_coef_int: 0.7553 - binary_accuracy: 0.9732 - val_loss: 0.6613 - val_binary_crossentropy: 0.2425 - val_jaccard_coef_int: 0.7601 - val_binary_accuracy: 0.9760
Epoch 31/50
100/100 [==============================] - 9s 88ms/step - loss: 0.6852 - binary_crossentropy: 0.2650 - jaccard_coef_int: 0.7605 - binary_accuracy: 0.9745 - val_loss: 0.6636 - val_binary_crossentropy: 0.2370 - val_jaccard_coef_int: 0.7519 - val_binary_accuracy: 0.9764
Epoch 32/50
100/100 [==============================] - 9s 88ms/step - loss: 0.6869 - binary_crossentropy: 0.2666 - jaccard_coef_int: 0.7615 - binary_accuracy: 0.9743 - val_loss: 0.9405 - val_binary_crossentropy: 0.3924 - val_jaccard_coef_int: 0.6949 - val_binary_accuracy: 0.9643
Epoch 33/50
100/100 [==============================] - 9s 89ms/step - loss: 0.6702 - binary_crossentropy: 0.2571 - jaccard_coef_int: 0.7634 - binary_accuracy: 0.9752 - val_

Epoch 7/50
100/100 [==============================] - 9s 88ms/step - loss: 1.3591 - binary_crossentropy: 0.1438 - jaccard_coef_int: 0.4882 - binary_accuracy: 0.9637 - val_loss: 1.1384 - val_binary_crossentropy: 0.1170 - val_jaccard_coef_int: 0.5736 - val_binary_accuracy: 0.9677
Epoch 8/50
100/100 [==============================] - 9s 88ms/step - loss: 1.2798 - binary_crossentropy: 0.1429 - jaccard_coef_int: 0.5099 - binary_accuracy: 0.9654 - val_loss: 1.1553 - val_binary_crossentropy: 0.1341 - val_jaccard_coef_int: 0.5704 - val_binary_accuracy: 0.9662
Epoch 9/50
100/100 [==============================] - 9s 86ms/step - loss: 1.2088 - binary_crossentropy: 0.1421 - jaccard_coef_int: 0.5295 - binary_accuracy: 0.9671 - val_loss: 0.9527 - val_binary_crossentropy: 0.1011 - val_jaccard_coef_int: 0.6281 - val_binary_accuracy: 0.9727
Epoch 10/50
100/100 [==============================] - 9s 88ms/step - loss: 1.1302 - binary_crossentropy: 0.1349 - jaccard_coef_int: 0.5500 - binary_accuracy: 0.96

...

Epoch 12/50
100/100 [==============================] - 9s 89ms/step - loss: 1.3929 - binary_crossentropy: 0.2979 - jaccard_coef_int: 0.5493 - binary_accuracy: 0.9522 - val_loss: 2.6793 - val_binary_crossentropy: 0.7098 - val_jaccard_coef_int: 0.2247 - val_binary_accuracy: 0.9259
Epoch 13/50
100/100 [==============================] - 9s 89ms/step - loss: 1.3347 - binary_crossentropy: 0.2834 - jaccard_coef_int: 0.5633 - binary_accuracy: 0.9543 - val_loss: 1.2885 - val_binary_crossentropy: 0.2603 - val_jaccard_coef_int: 0.5675 - val_binary_accuracy: 0.9566
Epoch 14/50
100/100 [==============================] - 9s 87ms/step - loss: 1.3172 - binary_crossentropy: 0.2817 - jaccard_coef_int: 0.5684 - binary_accuracy: 0.9548 - val_loss: 1.2148 - val_binary_crossentropy: 0.2367 - val_jaccard_coef_int: 0.5627 - val_binary_accuracy: 0.9614
Epoch 15/50
100/100 [==============================] - 9s 89ms/step - loss: 1.3282 - binary_crossentropy: 0.2916 - jaccard_coef_int: 0.5663 - binary_accuracy: 0

100/100 [==============================] - 9s 89ms/step - loss: 0.7372 - binary_crossentropy: 0.1448 - jaccard_coef_int: 0.7350 - binary_accuracy: 0.9744 - val_loss: 0.5563 - val_binary_crossentropy: 0.0922 - val_jaccard_coef_int: 0.7846 - val_binary_accuracy: 0.9818
Epoch 42/50
100/100 [==============================] - 9s 88ms/step - loss: 0.7939 - binary_crossentropy: 0.1660 - jaccard_coef_int: 0.7172 - binary_accuracy: 0.9724 - val_loss: 0.8217 - val_binary_crossentropy: 0.1512 - val_jaccard_coef_int: 0.6919 - val_binary_accuracy: 0.9722
Epoch 43/50
100/100 [==============================] - 9s 87ms/step - loss: 0.7120 - binary_crossentropy: 0.1394 - jaccard_coef_int: 0.7415 - binary_accuracy: 0.9754 - val_loss: 0.9732 - val_binary_crossentropy: 0.2083 - val_jaccard_coef_int: 0.6368 - val_binary_accuracy: 0.9678
Epoch 44/50
100/100 [==============================] - 9s 89ms/step - loss: 0.7201 - binary_crossentropy: 0.1458 - jaccard_coef_int: 0.7414 - binary_accuracy: 0.9748 - val_

Epoch 18/50
100/100 [==============================] - 8s 85ms/step - loss: 0.9792 - binary_crossentropy: 0.0762 - jaccard_coef_int: 0.6650 - binary_accuracy: 0.9820 - val_loss: 1.4280 - val_binary_crossentropy: 0.1605 - val_jaccard_coef_int: 0.4109 - val_binary_accuracy: 0.9779
Epoch 19/50
100/100 [==============================] - 9s 86ms/step - loss: 0.9558 - binary_crossentropy: 0.0758 - jaccard_coef_int: 0.6686 - binary_accuracy: 0.9822 - val_loss: 0.9193 - val_binary_crossentropy: 0.0392 - val_jaccard_coef_int: 0.6321 - val_binary_accuracy: 0.9850
Epoch 20/50
100/100 [==============================] - 9s 85ms/step - loss: 0.9331 - binary_crossentropy: 0.0744 - jaccard_coef_int: 0.6739 - binary_accuracy: 0.9823 - val_loss: 1.0041 - val_binary_crossentropy: 0.0588 - val_jaccard_coef_int: 0.5768 - val_binary_accuracy: 0.9833
Epoch 21/50
100/100 [==============================] - 9s 87ms/step - loss: 0.8898 - binary_crossentropy: 0.0708 - jaccard_coef_int: 0.6857 - binary_accuracy: 0

100/100 [==============================] - 9s 85ms/step - loss: 0.4555 - binary_crossentropy: 0.0453 - jaccard_coef_int: 0.8006 - binary_accuracy: 0.9900 - val_loss: 0.3548 - val_binary_crossentropy: 0.0235 - val_jaccard_coef_int: 0.8197 - val_binary_accuracy: 0.9943
Epoch 48/50
100/100 [==============================] - 8s 85ms/step - loss: 0.4429 - binary_crossentropy: 0.0436 - jaccard_coef_int: 0.8031 - binary_accuracy: 0.9904 - val_loss: 0.3591 - val_binary_crossentropy: 0.0219 - val_jaccard_coef_int: 0.8156 - val_binary_accuracy: 0.9940
Epoch 49/50
100/100 [==============================] - 9s 86ms/step - loss: 0.4410 - binary_crossentropy: 0.0442 - jaccard_coef_int: 0.8022 - binary_accuracy: 0.9904 - val_loss: 0.4412 - val_binary_crossentropy: 0.0262 - val_jaccard_coef_int: 0.7721 - val_binary_accuracy: 0.9920
Epoch 50/50
100/100 [==============================] - 8s 84ms/step - loss: 0.4303 - binary_crossentropy: 0.0442 - jaccard_coef_int: 0.8074 - binary_accuracy: 0.9905 - val_

100/100 [==============================] - 10s 95ms/step - loss: 0.9956 - binary_crossentropy: 0.2474 - jaccard_coef_int: 0.7081 - binary_accuracy: 0.9607 - val_loss: 0.7934 - val_binary_crossentropy: 0.1845 - val_jaccard_coef_int: 0.7811 - val_binary_accuracy: 0.9675
Epoch 25/50
100/100 [==============================] - 9s 86ms/step - loss: 1.0000 - binary_crossentropy: 0.2550 - jaccard_coef_int: 0.7086 - binary_accuracy: 0.9603 - val_loss: 0.8373 - val_binary_crossentropy: 0.2150 - val_jaccard_coef_int: 0.7568 - val_binary_accuracy: 0.9665
Epoch 26/50
100/100 [==============================] - 9s 94ms/step - loss: 0.9370 - binary_crossentropy: 0.2350 - jaccard_coef_int: 0.7247 - binary_accuracy: 0.9628 - val_loss: 1.6550 - val_binary_crossentropy: 0.5387 - val_jaccard_coef_int: 0.5758 - val_binary_accuracy: 0.9315
Epoch 27/50
100/100 [==============================] - 10s 96ms/step - loss: 0.9014 - binary_crossentropy: 0.2252 - jaccard_coef_int: 0.7341 - binary_accuracy: 0.9643 - va